# Projeto Análise de Qualidade de Alimentos em Plantações Agrícola com IA

O objetivo é fazer o fine-tuningem um modelo Vision Transformer Pré-treinado  e  ajustá-lo  ao  nosso  próprio  caso  de  uso, a  fim  de  classificar  e  prever  a  qualidade  de alimentos  em  plantações  agrícolas. 

Com base em uma imagem de folha, o  objetivo desta tarefa é prever o tipo de doença (Mancha Angular e Ferrugem do Feijão), se houver.Os termos em inglês são Angular Leaf Spot eBean Rust.

Fonte dos dados: https://huggingface.co/datasets/beans

## 1. Instalando e carregando os pacotes

In [ ]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

In [ ]:
# Ocultas avisos do Tensorflow
%env TF_CPP_MIN_LOG_LEVEL=3

In [ ]:
# Instala Torch
!pip install -q torch

In [ ]:
# Instala Transformers
!pip install -q transformers

In [ ]:
# Instala datasets
!pip install -q datasets==2.11.0

In [ ]:
# Instala accelerate
!pip install accelerate

In [ ]:
# Imports

from PIL import Image
import requests
import accelerate
import torch
import datasets
import transformers
import numpy as np
from datasets import load_dataset, load_metric
from transformers import ViTFeatureExtractor, ViTForImageClassification
from transformers import TrainingArguments
from transformers import Trainer
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Projeto Análise Imagens Agrícolas com IA" --iversions

## 2. Aplicando o Fine-Tunning ao Modelo
Pré-processamento para processar os dados do modelo

### 2.1 Carga do dataset no disco

In [ ]:
# Carrega os dados
dados = load_dataset('beans')

In [ ]:
# Visualiza os dados
print(dados)

In [ ]:
# Extrai os labels
labels = dados['train'].features['labels']

# Visualiza os dados
print(labels)

### 2.2 Aplicando o ViT Feature Extractor para Processar as Imagens

O ViT Feature Extractor serve para transformar imagens de entrada em representações vetoriais de alto nível que podem ser usadas para uma variedade de tarefas além da classificação de imagens.

In [ ]:
# Repositório do ViT pré-treinado
repo_id = 'google/vit-base-patch16-224-in21k'

In [ ]:
# Importa o ViTFeatureExtractor
feature_extractor = ViTFeatureExtractor.from_pretrained(repo_id)

In [ ]:
# Visualiza o FeatureExtractor
print(feature_extractor)

In [ ]:
# Função para o mapeamento de lotes de imagens e alpicação do ViTFeatureExtractor
def transform(example_batch):
    inputs = feature_extractor([x for x in example_batch['image']], return_tensors = 'pt')
    inputs['labels'] = example_batch['labels']
    return inputs

In [ ]:
# Prepara dos dados
prepared_data = dados.with_transform(transform)

In [ ]:
# Visualiza um exemplo para conferir se a função está correta
prepared_data['train'][0:2]

In [ ]:
# Função para combinar as amostras
# A função combina múltiplas amostras em um único lote para o processamento do Pytorch

def collate_fn(batch):
    
    return{'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
          'labels': torch.tensor([x['labels'] for x in batch])}

## 3. Construção do Módulo de Treino do ViT

In [ ]:
# Métrica do modelo
metric = load_metric('accuracy')

In [ ]:
# Cálculo da métrica
def compute_metrics(prediction):
    return metric.compute(predictions = np.armax(prediction.predictions, axis = 1),
                         references = prediction.label_ids)

In [ ]:
# Copia os labels
labels = dados['train'].features['labels'].names

In [ ]:
# Visualiza os labels
print(labels)

In [ ]:
# Importa o modelo ViTForImageClassification indicandos os novos labels que serão usados
modelo = ViTForImageClassification.from_pretrained(repo_id,
                                                  num_labels = len(labels),
                                                  id2label = {str(i):c for i, c in enumerate(labels)},
                                                  label2id = {c:str(i) for i, c in enumerate(labels)})

Esta mensagem acima está dizendo que você está inicializando um modelo ViTForImageClassification (um modelo de classificação de imagem Vision Transformer) a partir de um modelo pré-treinado (google/vit-base-patch16-224-in21k). No entanto, nem todos os pesos do modelo pré-treinado estão sendo usados na inicialização e alguns pesos do modelo ViTForImageClassification estão sendo inicializados do zero.

Há dois pontos principais aqui:

Alguns pesos do checkpoint do modelo ('pooler.dense.bias', 'pooler.dense.weight') não foram utilizados na inicialização do ViTForImageClassification. Isso pode ser esperado se você estiver inicializando o ViTForImageClassification a partir do checkpoint de um modelo treinado em outra tarefa ou com outra arquitetura. Se o modelo pré-treinado fosse exatamente idêntico à arquitetura que você está inicializando, você esperaria que todos os pesos fossem usados, e a mensagem informaria que algo está errado se isso não acontecesse.

Alguns pesos do ViTForImageClassification ('classifier.bias', 'classifier.weight') não foram inicializados a partir do checkpoint do modelo e foram recém-inicializados. Isso significa que esses componentes específicos do modelo não receberam pesos do modelo pré-treinado e, em vez disso, foram inicializados, provavelmente com alguma forma de inicialização aleatória.

A mensagem termina sugerindo que você provavelmente deve treinar (ou seja, fazer um "fine-tuning") este modelo em uma tarefa antes de usá-lo para predições e inferências. Isso ocorre porque, embora o modelo tenha sido parcialmente inicializado com pesos de um modelo pré-treinado, ele ainda tem alguns pesos que foram inicializados aleatoriamente e, portanto, precisam ser ajustados para a tarefa específica que você deseja resolver.

In [ ]:
# Argumentos de treino
training_args = TrainingArguments(output_dir = "resultados",
                                  evaluation_strategy = 'steps',
                                  num_train_epochs = 4,
                                  learning_rate = 2e-4,
                                  remove_unused_columns = False,
                                  load_best_model_at_end = True)

In [ ]:
# Trainer
trainer = Trainer(model = modelo,
                  args = training_args,
                  data_collator = collate_fn,
                  compute_metrics = compute_metrics,
                  train_dataset = prepared_data['train'],
                  eval_dataset = prepared_data['validation'],
                  tokenizer = feature_extractor)

## 4. Treino do modelo

In [ ]:
%%time
train_results = trainer.train()

In [ ]:
# Salva o modelo em disco
trainer.save_model('modelos')

In [ ]:
# Log das métricas
trainer.log_metrics('train', train_results.metrics) env

In [ ]:
# Salva as métricas
trainer.save_metrics('train', train_results.metrics)

## 5. Avaliação do modelo

In [ ]:
# Avaliação do modelo 
metrics = trainer.evaluate(prepared_data['validation'])
trainer.log_metrics('eval', metrics)
trainer.save_metrics('eval', metrics)

## 6. Deploy do modelo

In [ ]:
# URL de uma imagem (experimente outras imagens)
url = 'https://www.greenlife.co.ke/wp-content/uploads/2022/04/disease_bean_angular_leaf_spot.jpg'

In [ ]:
# Carrega a imagem
image = Image.open(requests.get(url, stream = True).raw)

In [ ]:
# Aplica o extrator
inputs = feature_extractor(images = image, return_tensors = "pt")

In [ ]:
# Coloca a imagem no mesmo device do modelo
inputs = {name: tensor.to(trainer.args.device) for name, tensor in inputs.items()}

In [ ]:
# Visualiza os inputs
inputs

In [ ]:
# Coloca o modelo em modo de avaliação
trainer.model.eval() 

# Desliga os gradientes para a inferência
with torch.no_grad():  
    outputs = trainer.model(**inputs)

In [ ]:
# Extrai os logits
logits = outputs.logits  

In [ ]:
# Extrai o logit de maior valor para a imagem
class_index = logits.argmax()

In [ ]:
# Extrai o valor que desejamos do tensor
valor = class_index.item()
valor

In [ ]:
# Define o mapeamento original (está na documentação do dataset bean)
mapping = {
  "angular_leaf_spot": 0,
  "bean_rust": 1,
  "healthy": 2,
}

In [ ]:
# Cria um mapeamento reverso
reverse_mapping = {v: k for k, v in mapping.items()}
reverse_mapping

In [ ]:
# Usa o mapeamento reverso para obter o nome da classe
class_name = reverse_mapping.get(valor)

In [ ]:
print("A imagem foi classificada como:", class_name)